In [29]:
import os
from mpetools import IslandTime
import numpy as np
import json
import requests
from requests.auth import HTTPBasicAuth
import matplotlib.pyplot as plt
from osgeo import gdal
import pyproj
from skimage.filters import threshold_multiotsu
from skimage import measure
from xml.dom import minidom
import datetime
import scipy.signal
import scipy.stats
from celluloid import Camera

In [2]:
island = 'Vodamulaa'
country = 'Maldives'
island_info = IslandTime.retrieve_island_info(island, country, verbose=False)

In [3]:
path_data = os.path.join(os.getcwd(), 'data', 'coastsat_data', '{}_{}'.format(island, country), 'PSScene', '{}_{}_2017_psscene_analytic_udm2'.format(island, country), 'PSScene')

In [39]:
listdir = os.listdir(path_data)
listfile = [f for f in listdir if f.endswith('_AnalyticMS_clip.tif')]

for i in range(len(listfile))[:15]:

    first_file = listfile[i]
    corresponding_xml_file = first_file.replace('_AnalyticMS_clip.tif', '_AnalyticMS_metadata_clip.xml')

    str_date = first_file.split('_')[0]
    year = str_date[:4]
    month = str_date[4:6]
    day = str_date[6:8]
    datetime.datetime(int(year), int(month), int(day))

    xmldoc = minidom.parse(os.path.join(path_data, corresponding_xml_file))
    nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")

    # XML parser refers to bands by numbers 1-4
    coeffs = {}
    for node in nodes:
        bn = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
        if bn in ['1', '2', '3', '4']:
            i = int(bn)
            value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
            coeffs[i] = float(value)
        
    img = gdal.Open(os.path.join(path_data, first_file), gdal.GA_ReadOnly)
    blue = img.GetRasterBand(1).ReadAsArray()
    green = img.GetRasterBand(2).ReadAsArray()
    red = img.GetRasterBand(3).ReadAsArray()
    nir = img.GetRasterBand(4).ReadAsArray()

    # Multiply by corresponding coefficients
    redc = red * coeffs[3]
    nirc = nir * coeffs[4]
    greenc = green * coeffs[2]
    bluec = blue * coeffs[1]
    rgb = np.dstack((redc, greenc, bluec))

    ndvi = (nirc.astype(float) - redc.astype(float)) / (nirc + redc)
    ndwi = (greenc.astype(float) - nirc.astype(float)) / (greenc + nirc)

    imgg = ndwi.copy()

    # First Otsu to remove water
    f_otsu = threshold_multiotsu(ndwi[~np.isnan(imgg)], classes=2)

    # Distribution of ndwi
    ndwi_cut = imgg[~np.isnan(imgg)] #[imgg[~np.isnan(imgg)] > f_otsu[0]]

    hist = np.histogram(ndwi_cut, bins=150)
    X = np.linspace(min(ndwi_cut), max(ndwi_cut), 10000)
    #plt.hist(ndwi_cut, bins=250, density=True)
    hist_dist = scipy.stats.rv_histogram(hist)
    sigma = 50.  # Adjust the standard deviation as needed
    smoothed_signal = scipy.ndimage.gaussian_filter(hist_dist.pdf(X), sigma)
    #plt.plot(X, hist_dist.pdf(X))
    #plt.plot(X, smoothed_signal)

    peaks, _ = scipy.signal.find_peaks(smoothed_signal, height=1)
    print(peaks)

    if len(peaks) < 5:
        otsu = threshold_multiotsu(ndwi_cut, classes=len(peaks)+1)
    else:
        otsu = threshold_multiotsu(ndwi_cut, classes=5)
   
    plt.figure()
    plt.hist(ndwi_cut, bins=100)
    for peak in peaks:
        plt.axvline(X[peak], color='g')
    for i in range(len(otsu)):
        plt.axvline(otsu[i], color='r')
    plt.title('{}-{}-{}'.format(year, month, day))
    plt.show()
    

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))  # 1 row, 2 columns
    for idx_t, t in enumerate(otsu):
        contours = measure.find_contours(imgg, t, fully_connected='high', positive_orientation='high')
        ax1.imshow(ndwi, cmap='gray')
        argm = np.argmax([len(c) for c in contours])
        for ii, cc in enumerate(contours):
            if ii == argm and len(cc) > 100:
                #if idx_t == 0:
                ax1.plot(cc[:, 1], cc[:, 0])
                #elif idx_t == 1:
                    #ax1.plot(cc[:, 1], cc[:, 0], color='k')
                #else:
                    #ax1.plot(cc[:, 1], cc[:, 0], color='b')
    ax1.set_title('{}-{}-{}'.format(year, month, day))
    ax2.imshow(rgb)
    ax2.set_title('RGB')
    plt.show()

[7533 8185 9531]
[7393 7818 8493 9330]
[7372 8151 9265]
[7794 8885 9515]
[7228 8205 9044]
[7143 8200]
[7548 9558]
[7536 8411 9496]
[7441 8258 9428]
[6139 6942 7525 7881 8504 8767 9000]
[7631 9534]
[7046 9060 9640]
[7479 8346 9505]
[7197 8605 9481]
[6764 7672 9395]


In [19]:
img = gdal.Open(os.path.join(path_data, first_file), gdal.GA_ReadOnly)
blue = img.GetRasterBand(1).ReadAsArray()
green = img.GetRasterBand(2).ReadAsArray()
red = img.GetRasterBand(3).ReadAsArray()
nir = img.GetRasterBand(4).ReadAsArray()

# Get geospatial information
geotransform = img.GetGeoTransform()
projection = img.GetProjection()

# Extract geospatial parameters
x_origin = geotransform[0]
y_origin = geotransform[3]
pixel_width = geotransform[1]
pixel_height = geotransform[5]

# Create arrays of longitude and latitude coordinates
width, height = img.RasterXSize, img.RasterYSize
x_coords = np.arange(width) * pixel_width + x_origin
y_coords = np.arange(height) * pixel_height + y_origin

# Create a meshgrid of longitude and latitude
lon, lat = np.meshgrid(x_coords, y_coords)

src_crs = pyproj.CRS('EPSG:{}'.format(pyproj.CRS.from_string(projection).to_epsg()))
tgt_crs = pyproj.CRS('EPSG:4326')

# Create a transformer
transformer = pyproj.Transformer.from_crs(src_crs, tgt_crs, always_xy=True)

# Reproject the data
x_reprojected, y_reprojected = transformer.transform(lon, lat)

# Create a figure and plot the GeoTIFF data
plt.figure(figsize=(10, 10))
plt.imshow(nir, extent=[x_reprojected.min(), x_reprojected.max(), y_reprojected.min(), y_reprojected.max()], cmap='viridis')
plt.scatter(island_info['spatial_reference']['longitude'], island_info['spatial_reference']['latitude'])

# You can add more customization to the plot, such as labels, a colorbar, etc.
plt.title("GeoTIFF Image with Lat-Lon Coordinates")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.colorbar(label="Data Values")

# Show the plot
plt.show()